In [27]:
from PIL import Image
import cv2
import torch
import math 
import function.utils_rotate as utils_rotate
from IPython.display import display
import os
import function.helper as helper
from ultralytics import YOLO

In [28]:
# Load YOLOv8 models
# Please ensure these files exist in your 'model/' directory after training
try:
    yolo_LP_detect = YOLO('model/LP_detector.pt')
    yolo_license_plate = YOLO('model/LP_ocr.pt')
except Exception as e:
    print(f"Error loading models: {e}")
    print("Please ensure you have trained the models and copied 'best.pt' to 'model/LP_detector.pt' and 'model/LP_ocr.pt'")

In [29]:
# enter image path here
img_file = "test_image/119.jpg"
img = cv2.imread(img_file)

# YOLOv8 inference
results = yolo_LP_detect.predict(img, imgsz=640, conf=0.5)
plates = results[0].boxes.data.tolist()

In [30]:
list_read_plates = set()
count = 0

if len(plates) == 0:
    # Pass the YOLOv8 model object directly
    lp = helper.read_plate(yolo_license_plate, img)
    if lp != "unknown":
        list_read_plates.add(lp)
else:
    for plate in plates:
        flag = 0
        x = int(plate[0]) # xmin
        y = int(plate[1]) # ymin
        w = int(plate[2] - plate[0]) # width
        h = int(plate[3] - plate[1]) # height
        
        crop_img = img[y:y+h, x:x+w]
        cv2.rectangle(img, (int(plate[0]), int(plate[1])), (int(plate[2]), int(plate[3])), color = (0,0,255), thickness = 2)
        cv2.imwrite("crop.jpg", crop_img)
        rc_image = cv2.imread("crop.jpg")
        lp = ""
        count+=1
        for cc in range(0,2):
            for ct in range(0,2):
                # Pass the YOLOv8 model object directly
                lp = helper.read_plate(yolo_license_plate, utils_rotate.deskew(crop_img, cc, ct))
                if lp != "unknown":
                    list_read_plates.add(lp)
                    flag = 1
                    break
            if flag == 1:
                break

In [31]:
print(list_read_plates)
display(Image.open(img_file))
# display(Image.open("crop.jpg")) # Uncomment to see the cropped plate